In [1]:
using CSV, DataFrames, Turing, CategoricalArrays, StatsBase, StatsPlots, Random,
    ReverseDiff, Revise, RCall, LinearAlgebra
using OptimizationOptimJL, Distributions, ApproxFun, Serialization, Printf, DataFramesMeta,
    StatProfilerHTML, StatsFuns, OptimizationBBO, Printf

using Interact
includet("debughelpers.jl")
includet("fithelpers.jl")
includet("gen_inits.jl")
includet("models.jl")
includet("fitmodel1.jl")
includet("fitmodel2.jl")
## includet("fitmodel3.jl")


WebIO._IJuliaInit()

ERROR: syntax: misplaced assignment statement in "[a = [0.7, 2.3, 2.5, 0.1, -0.3, -0.3]; c = [2.5, 2.5, 2.5, 2.5, 2.5, 2.5]; d0 = [0.1, 0.1, 0.1, 0.1, 0.1, 0.1]; dscale = [0.3, 0.3, 0.5, 0.5, 0.3, 0.3]; logconst = -5; [1.5]; [5]]" around /home/donkon/Documents/GermanMigration/gen_inits.jl:25
Stacktrace:
 [1] top-level scope
   @ none:1
in expression starting at /home/donkon/Documents/GermanMigration/gen_inits.jl:15


In [2]:
## Load the data and create the model object

##Random.seed!(20240719)


#optis = CSV.read("./fitted_models/opti" * model * ".csv", DataFrame)
optis = nothing

dt = load_flows()
dt.fromdist = categorical(dt.fromdist)
dt.todist = categorical(dt.todist)
dt.agegroup = categorical(dt.agegroup)
levels!(dt.agegroup,["below18","18-25","25-30","30-50","50-65","above65"])
rename!(dt, Dict(:dist => :distance))

## Create a districts file which has distcode, pop, density, xcoord, ycoord and save it in the data directory
dists = CSV.read("./data/districts.csv",DataFrame)
dists.distcode = categorical(dists.distcode)

distances = [norm([dists.xcoord[i]-dists.xcoord[j],dists.ycoord[i]-dists.ycoord[j]]) for i in 1:nrow(dists) , j in 1:nrow(dists) if i != j]
meddist = median(distances)
distances = nothing ## free the memory

popgerm = sum(dists.pop)

Nages = length(levels(dt.agegroup))
Ndist = nrow(dists)

netactual = calcnet(dt.flows,
            levelcode.(dt.fromdist),
            levelcode.(dt.todist),
            levelcode.(dt.agegroup),
            Nages,
            Ndist)
Ncoefs = 36

dtsmall = dt[rand(Bernoulli(.02),nrow(dt)),:]
dtsmall = dtsmall[dtsmall.flows .> 0.0,:]

model3 = migration3(dtsmall.flows,sum(dtsmall.flows),levelcode.(dtsmall.fromdist),levelcode.(dtsmall.todist),
                        dtsmall.frompop,dtsmall.topop, popgerm, dtsmall.distance, levelcode.(dtsmall.agegroup),Nages,
                        dists.xcoord,dists.ycoord,dists.density,dists.pop,Ndist,meddist,netactual,Ncoefs)



DynamicPPL.Model{typeof(migration3), (:flows, :allmoves, :fromdist, :todist, :frompop, :topop, :popgerm, :distance, :agegroup, :Nages, :xcoord, :ycoord, :density, :distpop, :Ndist, :meddist, :netactual, :ncoefs), (), (), Tuple{Vector{Int64}, Int64, Vector{Int64}, Vector{Int64}, Vector{Float64}, Vector{Float64}, Float64, Vector{Int64}, Vector{Int64}, Int64, Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Float64}, Int64, Float64, Matrix{Int64}, Int64}, Tuple{}, DynamicPPL.DefaultContext}(migration3, (flows = [14, 7, 1, 7, 1, 4, 4, 1, 1, 1  …  2, 1, 2, 1, 8, 16, 7, 2, 1, 7], allmoves = 39519, fromdist = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  401, 401, 401, 401, 401, 401, 401, 401, 401, 401], todist = [2, 6, 6, 10, 22, 40, 53, 93, 136, 175  …  198, 207, 263, 342, 354, 371, 371, 388, 398, 400], frompop = [11.765, 17.061, 11.765, 11.765, 17.061, 17.061, 17.061, 8.839, 15.804, 8.839  …  23.222, 23.222, 19.435, 3.217, 27.031, 3.275, 23.222, 27.031, 23.222, 3.217], topop = [32.068, 43.409

# Find Inits for model3

The goal will be to load in the data, and find the initial conditions for model 3 that are a good starting point in terms of the parameters
* a
* c
* d0
* dscale
* logisticconst

In [9]:


@manipulate for a1 = -10.0:.1:10.0,
                a2 = -10.0:.1:10.0,
                a3 = -10.0:.1:10.0,
                a4 = -10.0:.1:10.0,
                a5 = -10.0:.1:10.0,
                a6 = -10.0:.1:10.0,
                c = 0.01:.1:10.0,
                d0 = 0.001:.01:2.0,
                dscale = 0.05:.05:2.5,
                neterr = 0.1:.1:2.0,
                logisticconst = -10:.1:10
print(d0)
    as = [.7, 2.3, 2.5, .1, -.3, -.3]
   ##as = [-10, -10, -10, -10, -10, -.3] 
    cs = [2.5, 2.5, 2.5, 2.5, 2.5, 2.5]
    d0s = [.1, .1, .1, .1, .1, .1]
    dscales = [.3, .3, .5, .5, .3, .3]
    logconst = -5
   ##as = fill(-2, Nages)
   logconst = 4.5
    vals = [as;
            cs;
            d0s;   
            dscales;
            [neterr, logconst];
            fill(0.0,Nages); #kd
            fill(0.0,Nages*Ncoefs)
            ]
    names = [["a[$i]" for i in 1:Nages];
             ["c[$i]" for i in 1:Nages];
             ["d0[$i]" for i in 1:Nages];
             ["dscale[$i]" for i in 1:Nages];
             ["neterr","logisticconst"];
             ["kd[$i]" for i in 1:Nages];
             ["desirecoefs[$i]" for i in 1:(Ncoefs*Nages)];
             ]
    
    (preds,netflows) = generated_quantities(model3,vals,names)
    #@show preds
    scatter(dtsmall.distance, log.(dtsmall.flows ./ preds),group=dtsmall.agegroup,alpha=0.2,ylim=(-4,4))
end


0.991

LoadError: MethodError: no method matching fill(::Float64, ::Vector{Any})

[0mClosest candidates are:
[0m  fill(::Any, [91m::Tuple{}[39m)
[0m[90m   @[39m [90mBase[39m [90m[4marray.jl:585[24m[39m
[0m  fill(::Any, [91m::Tuple{Union{Infinities.InfiniteCardinal{0}, Infinities.Infinity}}[39m)
[0m[90m   @[39m [35mInfiniteArrays[39m [90m~/.julia/packages/InfiniteArrays/yaXl7/src/[39m[90m[4minfarrays.jl:77[24m[39m
[0m  fill(::Any, [91m::Tuple{Union{Infinities.InfiniteCardinal{0}, Infinities.Infinity}, Union{Infinities.InfiniteCardinal{0}, Infinities.Infinity}}[39m)
[0m[90m   @[39m [35mInfiniteArrays[39m [90m~/.julia/packages/InfiniteArrays/yaXl7/src/[39m[90m[4minfarrays.jl:80[24m[39m
[0m  ...


In [4]:
Nages = 
        vals = [[a1,a2,a3,a4,a5,a6];
            fill(c,Nages);
            fill(d0,Nages);
            fill(dscale,Nages);
            [neterr, logisticconst];
            fill(0.0,Nages); #kd
            fill(0.0,Nages*Ncoefs)
            ]
    names = [["a[$i]" for i in 1:Nages];
             ["c[$i]" for i in 1:Nages];
             ["d0[$i]" for i in 1:Nages];
             ["dscale[$i]" for i in 1:Nages];
             ["neterr","logisticconst"];
             ["kd[$i]" for i in 1:Nages];
             ["desirecoefs[$i]" for i in 1:(Ncoefs*Nages)];
             ]
    
    (preds,netflows) = generated_quantities(model3,vals,names)
    #@show preds
    scatter(dtsmall.distance, log.(dtsmall.flows ./ preds),group=dtsmall.agegroup,alpha=0.2,ylim=(-4,4))
end


LoadError: MethodError: no method matching (::Colon)(::Int64, ::Vector{Any})

[0mClosest candidates are:
[0m  (::Colon)(::Real, ::Any, [91m::Union{Infinities.InfiniteCardinal{0}, Infinities.Infinity}[39m)
[0m[90m   @[39m [35mInfiniteArrays[39m [90m~/.julia/packages/InfiniteArrays/yaXl7/src/[39m[90m[4minfrange.jl:12[24m[39m
[0m  (::Colon)(::A, ::Any, [91m::C[39m) where {A<:Real, C<:Unitful.Quantity}
[0m[90m   @[39m [32mUnitful[39m [90m~/.julia/packages/Unitful/GYzMo/src/[39m[90m[4mrange.jl:86[24m[39m
[0m  (::Colon)(::T, ::Any, [91m::T[39m) where T<:Real
[0m[90m   @[39m [90mBase[39m [90m[4mrange.jl:50[24m[39m
[0m  ...
